# Logger

> A basic logging class

In [ ]:
# | default_exp client.Logger


In [ ]:
# | hide
from nbdev.showdoc import *
from fastcore.test import test_eq


In [ ]:
# | exporti
import datetime as dt

from typing import Optional, List
from dataclasses import dataclass

import traceback

from fastcore.basics import patch_to

In [ ]:
# | export


class Logger:
    """log class with user customizeable output method"""

    root_module: str
    app_name: str

    logs: List[dict]
    breadcrumb: Optional[list]

    entity_id: Optional[str]
    domo_instance: Optional[str]
    # function to call with write_logs method.
    output_fn: Optional[callable] = None

    def __init__(
        self,
        app_name: str,  # name of the app for grouping logs
        root_module: Optional[str] = "<module>",  # root module for stack trace
        output_fn: Optional[
            callable
        ] = None,  # function to call with write_logs method.
        entity_id: Optional[str] = None,
        domo_instance: Optional[str] = None,
    ):

        self.app_name = app_name
        self.output_fn = output_fn
        self.root_module = root_module
        self.logs = []
        self.breadcrumb = []
        self.domo_instance = domo_instance
        self.entity_id = entity_id

    def _add_crumb(self, crumb):
        if crumb not in self.breadcrumb:
            self.breadcrumb.append(crumb)

    def _remove_crumb(self, crumb):
        if crumb in self.breadcrumb:
            self.breadcrumb.remove(crumb)

## Logger traceback methods

Private methods for adding traceback details to logs


In [ ]:
# | export


@patch_to(Logger)
def _get_traceback(
    self :Logger,
    root_module: str = "<module>",
    num_stacks_to_drop=0,  # drop entries from the top of stack to exclude the functions that retrieve the traceback
) -> [
    traceback.FrameSummary
]:  # returns a filtered list of FrameSummaries from traceback
    """method that retrieves traceback"""

    tb = traceback.extract_stack()

    # find the last module index
    module_index = 0
    for index, tb_line in enumerate(tb):
        function_name = tb_line[2]

        if function_name == root_module:
            module_index = index

    if num_stacks_to_drop == 0:
        return tb[module_index:]

    return tb[module_index:-num_stacks_to_drop]

In [ ]:
show_doc(Logger._get_traceback)


---

[source](https://github.com/jaewilson07/domo_library/blob/main/domolibrary/Logger.py#L60){target="_blank" style="float:right; font-size:smaller"}

### Logger._get_traceback

>      Logger._get_traceback (root_module:str='<module>', num_stacks_to_drop=0)

method that retrieves traceback

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| root_module | str | <module> |  |
| num_stacks_to_drop | int | 0 | drop entries from the top of stack to exclude the functions that retrieve the traceback |
| **Returns** | **[<class 'traceback.FrameSummary'>]** |  |  |

In [ ]:
# | export


@dataclass
class TracebackDetails:
    """result of _get_traceback_details function"""

    function_name: str
    file_name: str
    function_trail: str


@patch_to(Logger)
def _get_traceback_details(
    self : Logger,
    traceback_list: [
        traceback.FrameSummary
    ],  # clean list of frame summaries from traceback (be sure to exclude frames from functions that retrieved the traceback)
) -> TracebackDetails:  # descriptive summary from the top of the traceback
    """returns TracebackDetails, for the entry at the top of the stack"""

    function_trail = " -> ".join([line[2] for line in traceback_list])

    function_name = traceback_list[-1][2]
    file_name = traceback_list[-1][0]

    return TracebackDetails(function_name, file_name, function_trail)

In [ ]:
show_doc(Logger._get_traceback_details)

---

[source](https://github.com/jaewilson07/domo_library/blob/main/domolibrary/Logger.py#L95){target="_blank" style="float:right; font-size:smaller"}

### Logger._get_traceback_details

>      Logger._get_traceback_details
>                                     (traceback_list:[<class'traceback.FrameSum
>                                     mary'>])

returns TracebackDetails, for the entry at the top of the stack

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| traceback_list | [<class 'traceback.FrameSummary'>] |  |
| **Returns** | **TracebackDetails** | **descriptive summary from the top of the traceback** |

### sample implementations of stack tracing methods


In [ ]:
import pandas as pd


def custom_write_logs_fn(logs):
    print("printing logs")
    return pd.DataFrame(logs)


logger = Logger(app_name="test", output_fn=custom_write_logs_fn)


def test_trace():
    return logger._get_traceback(num_stacks_to_drop=1)


# print traceback for the test_trace function
# notice with num_stacks_to_drop = 1 we exclude the _get_traceback function from the traceback
tb = test_trace()
print({"traceback function": [line[2] for line in tb]})


def test_get_details():
    tb = logger._get_traceback(num_stacks_to_drop=1)
    return logger._get_traceback_details(tb)


# print traceback details for test_get_details function
test_get_details().__dict__

{'traceback function': ['<module>', 'test_trace']}


{'function_name': 'test_get_details',
 'file_name': '/tmp/ipykernel_26360/2175548989.py',
 'function_trail': '<module> -> test_get_details'}

In [ ]:
# assert that the result of test_trace is of type FrameSummary
test_eq(type(tb[0]), traceback.FrameSummary)

## Logger logging methods


In [ ]:
# | export
@patch_to(Logger)
def _add_log(
    self :Logger, message: str, type_str: str, 
    debug_log : bool = False,
    num_stacks_to_drop=3, entity_id: Optional[str] = None, domo_instance: Optional[str] = None
) -> dict:
    """internal method to append message to log"""

    traceback_list = self._get_traceback(num_stacks_to_drop=num_stacks_to_drop)
    
    if debug_log:
        print({"num_stacks_to_drop": num_stacks_to_drop})
        print([tb_line[2] for tb_line in traceback_list])

    new_row = {
        "date_time": dt.datetime.now(),
        "application": self.app_name,
        "log_type": type_str,
        "log_message": message,
        "breadcrumb": "->".join(self.breadcrumb),
        "domo_instance": domo_instance or self.domo_instance,
        "entity_id": entity_id or self.entity_id
    }

    traceback_details = self._get_traceback_details(traceback_list)

    new_row.update(
        {
            "function_name": traceback_details.function_name,
            "file_name": traceback_details.file_name,
            "function_trail": traceback_details.function_trail,
        }
    )

    if debug_log:
        print(new_row)

    self.logs.append(new_row)

    return new_row


@patch_to(Logger)
def log_info(self : Logger, message,
             entity_id: Optional[str] = None,
             domo_instance: Optional[str] = None,
             debug_log=False, num_stacks_to_drop=3):
    """log an informational message"""
    return self._add_log(
        message=message,
        entity_id=entity_id,
        domo_instance=domo_instance,
        type_str="Info",
        num_stacks_to_drop=num_stacks_to_drop,
        debug_log=debug_log,
    )


@patch_to(Logger)
def log_error(self: Logger, message,
              entity_id: Optional[str] = None,
              domo_instance: Optional[str] = None, debug_log=False, num_stacks_to_drop=3):
    """log an error message"""

    return self._add_log(
        message=message,
        entity_id = entity_id,
        domo_instance = domo_instance,
        type_str="Error",
        num_stacks_to_drop=num_stacks_to_drop,
        debug_log=debug_log,
    )


@patch_to(Logger)
def log_warning(self :Logger, message,
                entity_id: Optional[str] = None,
                domo_instance: Optional[str] = None, 
                debug_log = False,
                num_stacks_to_drop=3):
    """log a warning message"""

    return self._add_log(
        message=message,
        entity_id=entity_id,
        domo_instance=domo_instance,
        type_str="Warning",
        num_stacks_to_drop=num_stacks_to_drop,
        debug_log = debug_log,
    )


In [ ]:
logger = Logger(app_name="test", output_fn=custom_write_logs_fn)


def test_log():
    return logger.log_info("test the error returns type Info", debug_log=False)


test_log()


NameError: name 'Logger' is not defined

## Outputting Logs

During Logger instantiation, users can pass a function, `output_fn` which will be called with the `Logger.output_log` method


In [ ]:
# | export

@patch_to(Logger)
def output_log(self:Logger):
    """calls the user defined output function"""
    return self.output_fn(self.logs)


##### Sample implementation with a custom write_logs method


In [ ]:
import pandas as pd

def custom_write_logs_fn(logs):
    print("printing logs")
    return pd.DataFrame(logs)


logger = Logger(app_name="test", output_fn=custom_write_logs_fn)


def test_error():
    try:
        if 1 == 1:
            raise Exception("random error")

    except Exception as e:
        logger.log_error(e)


def double_test():
    test_error()


# record first error
test_error()

# records second error nested inside double_test()
double_test()

logger.output_log()


printing logs


,date_time,application,log_type,log_message,breadcrumb,domo_instance,entity_id,function_name,file_name,function_trail
0,2023-01-17 16:06:56.921359,test,Error,random error,,None,None,test_error,/tmp/ipykernel_26360/218693317.py,<module> -> test_error
1,2023-01-17 16:06:56.921790,test,Error,random error,,None,None,test_error,/tmp/ipykernel_26360/218693317.py,<module> -> double_test -> test_error


In [ ]:
# | hide
# test that custom_write_logs_fn is stored in the logger as output_fn.
test_eq(logger.output_fn, custom_write_logs_fn)

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()